# Test visualiser module 

Tests for `visualiser.pydeck_routes`. The purpose is to convert arcs and required arcs dataframe into geo-pandas and json data frames, to be visualised using kepler.gl. Two versions are created. A high-level version that only shows the final routes. And a detailed version, showing the different elements of the routes.

The following board steps have to take place:

 1. Take the solution file, and convert it to path routes.
 2. Add the required path attributes.
 
Break-down the route into it's key steps, and visualise each portion. Not yet sure how this can be completed...

## Test modules

Nice magic that automatically reimports external modules that changed (don't have to restart the kernal) and making sure the source package path is accessible:

In [16]:
%load_ext autoreload
%autoreload 2

import os
import sys
import importlib
sys.path.insert(0, os.path.abspath('../../'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## General packages to use

In [17]:
import pandas as pd
import osmnx as ox
import numpy as np
import pydeck as pdk
import seaborn as sns
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
import geopandas as gpd
from shapely import wkt

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Test data

The following test data, generated via `tests_osmnx_network_extract`, is used.

In [85]:
df_arcs = pd.read_csv('../test_osmnx_network_extract/output/test_processed_full_arcs.csv')
df_arcs_req = pd.read_csv('../test_osmnx_network_extract/output/test_processed_required_arcs.csv')
df_key_locations = pd.read_csv('../test_osmnx_network_extract/output/df_key_locations.csv')
solution = pd.read_csv('../test_osmnx_network_extract/output/test_sol.csv')
solution = solution.reset_index() # forces an index for after merge sorting
G = nx.read_gpickle('../test_osmnx_network_extract/output/test_graph_nx.pickle')

## Conversion

We add all the arc and node info:

In [117]:
traversals = solution['activity_type'].isin(['traversal', 'collect'])
solution_traversals = solution.loc[traversals].copy()
solution_traversals['arc_id'] = solution_traversals['arc_start_node'].astype(str) + '-' + solution_traversals['arc_end_node'].astype(str)
solution_traversals = pd.merge(solution_traversals, gdf_arcs[['arc_id', 'arc_id_orig']], how='left')
solution_traversals = pd.merge(solution_traversals, gdf_arcs[['arc_id_orig', 'geometry', 'name', 'highway']], how='left')
solution_keypoints = solution[~traversals]
solution_keypoints = pd.merge(solution_keypoints, df_key_locations[['geometry', 'highway', 'u']], how='left', left_on='arc_start_node', right_on='u')
solution_keypoints = solution_keypoints.drop(columns=['u'])
solution_traversals = solution_traversals.sort_values(['index'])
solution_full = pd.concat([solution_keypoints, solution_traversals])
solution_full = solution_full.sort_values(['index'])

We can also use speed formulas to add realistic time-columns:

In [118]:
offload_time = 60 * 60 # seconds
travel_time = 10 * 1000 / 3600 # m / s
service_time = 1 * 1000 / 3600 # m / s

time_df = pd.DataFrame.from_dict({'activity_type': ['offload', 
                                                    'traversal', 
                                                    'collect', 
                                                    'depot_start', 
                                                    'depot_end',
                                                    'arrive_if',
                                                    'depart_if'], 
                                  'activity_speed': [offload_time, 
                                                     travel_time, 
                                                     service_time, 
                                                     1, 
                                                     1,
                                                     1,
                                                     1]})

In [119]:
solution_full = pd.merge(solution_full, time_df, how='left')
solution_full = solution_full.sort_values(['index'])
solution_full['activity_duration'] = solution_full['activity_time'] / solution_full['activity_speed']
solution_full.loc[solution_full['activity_type'] == 'offload', 'activity_duration'] = solution_full.loc[solution_full['activity_type'] == 'offload']['activity_speed']
solution_full['t'] = solution_full.groupby(['route'])['activity_duration'].cumsum().round()
solution_full

,index,route,subroute,activity_id,arc_start_node,arc_end_node,activity_type,total_traversal_time_to_activity,activity_time,activity_demand,...,cum_time,geometry,highway,osmid,arc_id,arc_id_orig,name,activity_speed,activity_duration,t
0,0,0,0,0,5184767962,5184767962,depot_start,0,0,0,...,0,POINT (103.7496458 1.3696026),NaN,5.184768e+09,NaN,NaN,NaN,1.000000,0.00,0.0
1,1,0,0,187,5184767962,5184767961,traversal,0,148,0,...,148,"LINESTRING (103.7496458 1.3696026, 103.7501433...",service,NaN,5184767962-5184767961,5184767962-5184767961-0,NaN,2.777778,53.28,53.0
2,2,0,0,184,5184767961,5184767967,traversal,0,47,0,...,195,"LINESTRING (103.7497297 1.3686712, 103.7494919...",service,NaN,5184767961-5184767967,5184767961-5184767967-0,NaN,2.777778,16.92,70.0
3,3,0,0,50,5184767967,5014445207,traversal,0,29,0,...,224,"LINESTRING (103.7494203 1.368375, 103.7496173 ...",tertiary,NaN,5184767967-5014445207,5184767967-5014445207-0,Bukit Batok West Avenue 5,2.777778,10.44,81.0
4,4,0,0,87,5014445207,243856692,traversal,0,8,0,...,232,"LINESTRING (103.7496173 1.3681935, 103.7495683...",service,NaN,5014445207-243856692,5014445207-243856692-0,NaN,2.777778,2.88,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,365,0,11,13,243856675,5014445201,traversal,0,49,0,...,26233,"LINESTRING (103.7486394 1.3690803, 103.7489092...",tertiary,NaN,243856675-5014445201,243856675-5014445201-0,Bukit Batok West Avenue 5,2.777778,17.64,61398.0
366,366,0,11,27,5014445201,5184767967,traversal,0,67,0,...,26300,"LINESTRING (103.7489804 1.3687955, 103.7492802...",tertiary,NaN,5014445201-5184767967,5014445201-5184767967-0,Bukit Batok West Avenue 5,2.777778,24.12,61422.0
367,367,0,11,185,5184767967,5184767961,traversal,0,47,0,...,26347,"LINESTRING (103.7494203 1.368375, 103.7494919 ...",service,NaN,5184767967-5184767961,5184767967-5184767961-0,NaN,2.777778,16.92,61439.0
368,368,0,11,186,5184767961,5184767962,traversal,0,148,0,...,26495,"LINESTRING (103.7497297 1.3686712, 103.7501447...",service,NaN,5184767961-5184767962,5184767961-5184767962-0,NaN,2.777778,53.28,61493.0


To help unpack the route we need to add a time dimension. This can be anything, from a few hours to a few days.

In [120]:
solution_full['time'] = '2020-06-14 08:00:00'
solution_full['time'] = pd.to_datetime(solution_full['time']).dt.tz_localize(None).dt.tz_localize('Asia/Singapore')
solution_full['time_fake'] = solution_full['time'] + pd.to_timedelta(list(range(len(solution_full))), unit='m')
solution_full['time'] = solution_full['time'] + pd.to_timedelta(solution_full['t'], unit='s')
solution_full

,index,route,subroute,activity_id,arc_start_node,arc_end_node,activity_type,total_traversal_time_to_activity,activity_time,activity_demand,...,highway,osmid,arc_id,arc_id_orig,name,activity_speed,activity_duration,t,time,time_fake
0,0,0,0,0,5184767962,5184767962,depot_start,0,0,0,...,NaN,5.184768e+09,NaN,NaN,NaN,1.000000,0.00,0.0,2020-06-14 08:00:00+08:00,2020-06-14 08:00:00+08:00
1,1,0,0,187,5184767962,5184767961,traversal,0,148,0,...,service,NaN,5184767962-5184767961,5184767962-5184767961-0,NaN,2.777778,53.28,53.0,2020-06-14 08:00:53+08:00,2020-06-14 08:01:00+08:00
2,2,0,0,184,5184767961,5184767967,traversal,0,47,0,...,service,NaN,5184767961-5184767967,5184767961-5184767967-0,NaN,2.777778,16.92,70.0,2020-06-14 08:01:10+08:00,2020-06-14 08:02:00+08:00
3,3,0,0,50,5184767967,5014445207,traversal,0,29,0,...,tertiary,NaN,5184767967-5014445207,5184767967-5014445207-0,Bukit Batok West Avenue 5,2.777778,10.44,81.0,2020-06-14 08:01:21+08:00,2020-06-14 08:03:00+08:00
4,4,0,0,87,5014445207,243856692,traversal,0,8,0,...,service,NaN,5014445207-243856692,5014445207-243856692-0,NaN,2.777778,2.88,84.0,2020-06-14 08:01:24+08:00,2020-06-14 08:04:00+08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,365,0,11,13,243856675,5014445201,traversal,0,49,0,...,tertiary,NaN,243856675-5014445201,243856675-5014445201-0,Bukit Batok West Avenue 5,2.777778,17.64,61398.0,2020-06-15 01:03:18+08:00,2020-06-14 14:05:00+08:00
366,366,0,11,27,5014445201,5184767967,traversal,0,67,0,...,tertiary,NaN,5014445201-5184767967,5014445201-5184767967-0,Bukit Batok West Avenue 5,2.777778,24.12,61422.0,2020-06-15 01:03:42+08:00,2020-06-14 14:06:00+08:00
367,367,0,11,185,5184767967,5184767961,traversal,0,47,0,...,service,NaN,5184767967-5184767961,5184767967-5184767961-0,NaN,2.777778,16.92,61439.0,2020-06-15 01:03:59+08:00,2020-06-14 14:07:00+08:00
368,368,0,11,186,5184767961,5184767962,traversal,0,148,0,...,service,NaN,5184767961-5184767962,5184767961-5184767962-0,NaN,2.777778,53.28,61493.0,2020-06-15 01:04:53+08:00,2020-06-14 14:08:00+08:00


kepler.gl automatically interprets geojson fields, so this below conversion is not needed. The results can be stored straight as a csv, which helps alot, since the frames can contain lists.

### Simplify output

Not all outputs are needed:

In [129]:
solution_full_simp = solution_full[['index', 'route', 'subroute', 'activity_type', 'activity_time', 
                                    'activity_demand', 'cum_demand', 'cum_time', 'remaining_capacity', 'remaining_time', 'geometry', 'highway', 'name', 'time', 'time_fake']].copy()
solution_full_simp['activity_type'] = solution_full_simp['activity_type'].str.replace('depot_start', 'depot')
solution_full_simp = solution_full_simp.loc[~solution_full_simp['activity_type'].isin(['arrive_if', 'depart_if'])]
solution_full_simp['activity_type'] = solution_full_simp['activity_type'].str.replace('depot_end', 'depot')
solution_full_simp['activity_type'] = solution_full_simp['activity_type'].str.replace('arrive_if', 'offload')
solution_full_simp['activity_type'] = solution_full_simp['activity_type'].str.replace('depart_if', 'offload')
solution_full_simp = solution_full_simp.rename(columns={'route': 'Vehicle ID',
                                                         'subroute': 'Sub-route',
                                                        'activity_type': 'Activity',
                                                        'activity_time': 'Distance (m)',
                                                        'activity_demand': 'Demand (l)',
                                                        'cum_demand': 'Total demand collected (l)',
                                                        'remaining_capacity': 'Remaining capacity (l)',
                                                        'cum_time': 'Total distance travelled (m)',
                                                        'remaining_time': 'Remaining bettery capacity (m)',
                                                        'highway': 'Road-type',
                                                        'name': 'Road name',
                                                        'time': 'Time',
                                                        'time_fake': 'Dummy time (1 min to traverse a road)'})
solution_full_simp.to_csv("output/test_full_paths.csv", index=False)

## Testing the package

In [280]:
from visualise import CreateRoutesGeo
activities = ['offload', 'traversal', 'collect', 'depot_start', 'depot_end', 'arrive_if', 'depart_if']
speed = [5 * 60, 10 / 3.6, 1 / 3.6, 1, 1, 1, 1]
time_df = pd.DataFrame.from_dict({'activity_type': activities, 'activity_speed': speed})

In [281]:
create_sol_geo = CreateRoutesGeo(df_arcs, df_key_locations, solution)
create_sol_geo.prepare_solution()
create_sol_geo.add_custom_duration(time_df)
create_sol_geo.add_time_formatted(start_time='1985-01-01 00:00:00')
create_sol_geo.add_constant_duration_time(start_time='1985-01-01 00:00:00')
create_sol_geo.simplify_output(keep_additional=['time', 'time_const_dur'])
create_sol_geo.prettify_columns_dist()
create_sol_geo.write_solution("output/test_full_paths_comp.csv")

In [268]:
create_sol_geo.solution

,index,Vehicle ID,Sub-route,Activity,Distance (m),Demand (kg),Total demand collected on sub-route (kg),Total distance travelled (m),Remaining capacity (kg),Remaining distance capacity (m),geometry,Road-type,Name (road or location),Time,Time (constant activity duration)
0,0,0,0,depot,0,0,0,0,4,30000,POINT (103.7496458 1.3696026),NaN,NaN,1985-01-01 00:00:00,1985-01-01 00:00:00
1,1,0,0,traversal,148,0,0,148,4,29852,"LINESTRING (103.7496458 1.3696026, 103.7501433...",service,NaN,1985-01-01 00:00:53,1985-01-01 00:01:00
2,2,0,0,traversal,47,0,0,195,4,29805,"LINESTRING (103.7497297 1.3686712, 103.7494919...",service,NaN,1985-01-01 00:01:10,1985-01-01 00:02:00
3,3,0,0,traversal,29,0,0,224,4,29776,"LINESTRING (103.7494203 1.368375, 103.7496173 ...",tertiary,Bukit Batok West Avenue 5,1985-01-01 00:01:21,1985-01-01 00:03:00
4,4,0,0,traversal,8,0,0,232,4,29768,"LINESTRING (103.7496173 1.3681935, 103.7495683...",service,NaN,1985-01-01 00:01:24,1985-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,365,0,11,traversal,49,0,0,26233,4,3767,"LINESTRING (103.7486394 1.3690803, 103.7489092...",tertiary,Bukit Batok West Avenue 5,1985-01-01 17:03:18,1985-01-01 06:47:00
408,366,0,11,traversal,67,0,0,26300,4,3700,"LINESTRING (103.7489804 1.3687955, 103.7492802...",tertiary,Bukit Batok West Avenue 5,1985-01-01 17:03:42,1985-01-01 06:48:00
409,367,0,11,traversal,47,0,0,26347,4,3653,"LINESTRING (103.7494203 1.368375, 103.7494919 ...",service,NaN,1985-01-01 17:03:59,1985-01-01 06:49:00
410,368,0,11,traversal,148,0,0,26495,4,3505,"LINESTRING (103.7497297 1.3686712, 103.7501447...",service,NaN,1985-01-01 17:04:53,1985-01-01 06:50:00


### Offload animation

We add entries where the total offloaded at a location is assigned a value, that can be visualised. This can be appended to the solution data frame to animated time. This can be left is beta for now.

In [269]:
create_sol_geo_offloads = CreateRoutesGeo(df_arcs, df_key_locations, solution)
create_sol_geo_offloads.prepare_solution()
create_sol_geo_offloads.add_custom_duration(time_df)
create_sol_geo_offloads.add_time_formatted(start_time='1985-01-01 00:00:00')
create_sol_geo_offloads.add_constant_duration_time(start_time='1985-01-01 00:00:00')

In [270]:
offload = create_sol_geo_offloads.solution.loc[create_sol_geo_offloads.solution['activity_type'] == 'arrive_if'][['index', 'cum_demand']].copy()
offload['index'] = offload['index'] + 1
offload_facilities = create_sol_geo_offloads.solution.loc[create_sol_geo_offloads.solution['activity_type'] == 'offload']
offload_facilities = offload_facilities[['index', 'activity_type', 'geometry', 'time', 'time_const_dur', 'name']]
offload_facilities = pd.merge(offload_facilities, offload).rename(columns={'cum_demand': 'offloaded'})

In [271]:
offload_facilities['name'] = 'Bin center'

In [272]:
all_times = pd.DataFrame(create_sol_geo_offloads.solution['time']).sort_values('time')
offload_facilities = offload_facilities.groupby(['activity_type', 'geometry', 'time', 'name']).agg(offloaded = ('offloaded', 'sum')).reset_index()
offload_facilities = pd.merge(offload_facilities, all_times, how='right')
offload_facilities = offload_facilities.sort_values(['time'])

In [273]:
offload_facilities['offloaded'] = offload_facilities['offloaded'].fillna(0)
offload_facilities[['offloaded', 'activity_type', 'geometry', 'name']] = offload_facilities[['offloaded', 'activity_type', 'geometry', 'name']].fillna(method='ffill')
offload_facilities[['offloaded', 'activity_type', 'geometry', 'name']] = offload_facilities[['offloaded', 'activity_type', 'geometry', 'name']].fillna(method='bfill')

In [274]:
offload_facilities['offloaded'] = offload_facilities.groupby(['geometry', 'name'])['offloaded'].cumsum()

In [277]:
offload_facilities

,activity_type,geometry,time,name,offloaded
23,offload,POINT (103.7503558 1.3664194),1985-01-01 00:00:00,Bin center,0.0
24,offload,POINT (103.7503558 1.3664194),1985-01-01 00:00:53,Bin center,0.0
25,offload,POINT (103.7503558 1.3664194),1985-01-01 00:01:10,Bin center,0.0
26,offload,POINT (103.7503558 1.3664194),1985-01-01 00:01:21,Bin center,0.0
27,offload,POINT (103.7503558 1.3664194),1985-01-01 00:01:24,Bin center,0.0
...,...,...,...,...,...
407,offload,POINT (103.7503558 1.3664194),1985-01-01 17:03:18,Bin center,88.0
408,offload,POINT (103.7503558 1.3664194),1985-01-01 17:03:42,Bin center,88.0
409,offload,POINT (103.7503558 1.3664194),1985-01-01 17:03:59,Bin center,88.0
410,offload,POINT (103.7503558 1.3664194),1985-01-01 17:04:53,Bin center,88.0


In [276]:
offload_solution = pd.concat([offload_facilities, create_sol_geo_offloads.solution])
offload_solution.to_csv('output/solution_with_offloads.csv', index=False)